### Alter and update EMPIRE Input - Sets.xlsx

In [144]:
import pandas as pd
import numpy as np

In [145]:
NODES_TO_BE_REMOVED = ["Utsira Nord", "Sørlige Nordsjø I", "Sørlige Nordsjø II"]

#### Tab: Nodes

In [146]:
nodes_tab = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="Nodes")
nodes_tab

,Node
0,Austria
1,Bosnia H
2,Belgium
3,Bulgaria
4,Switzerland
5,Czech R
6,Germany
7,Denmark
8,Estonia
9,Spain


Update with new OW nodes

In [147]:
OW_nodes = pd.read_csv("Output/OW_nodes.csv")
OW_nodes = OW_nodes[["Node"]]
nodes_tab = pd.concat([nodes_tab[~nodes_tab["Node"].isin(NODES_TO_BE_REMOVED)], OW_nodes]).drop_duplicates().reset_index(drop=True)
nodes_tab

,Node
0,Austria
1,Bosnia H
2,Belgium
3,Bulgaria
4,Switzerland
...,...
62,Nordavind A
63,Nordavind D
64,Nordavind C
65,Vestavind E


#### Tab: Storage

Keep as is

#### Tab: Technology 

Keep as is

#### Tab: Generators

Keep as is

#### Tab: LineType

Keep as is

#### Tab: StorageOfNodes

Keep as is

#### Tab: DirectionalLines

In [148]:
directional_lines_tab = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="DirectionalLines", header=2)[["NodeFrom", "NodeTo"]]
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeFrom"].isin(NODES_TO_BE_REMOVED)]
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeTo"].isin(NODES_TO_BE_REMOVED)].reset_index(drop=True)
directional_lines_tab

,NodeFrom,NodeTo
0,Switzerland,Austria
1,Czech R,Austria
2,Germany,Austria
3,Hungary,Austria
4,Italy,Austria
...,...,...
339,Great Brit.,Energyhub EU
340,Netherlands,Energyhub EU
341,Germany,Energyhub EU
342,Denmark,Energyhub EU


Grab newly generated directional lines

In [149]:
df_dl = pd.read_csv("Output/directional_lines.csv")
df_dl

,FromNode,ToNode,lineLength in km,LineType
0,Moray Firth,Firth of Forth,215.667240,HVDC_Cable
1,Firth of Forth,Dogger Bank,333.632410,HVDC_Cable
2,Firth of Forth,Hornsea,387.595780,HVDC_Cable
3,Dogger Bank,Hornsea,120.257880,HVDC_Cable
4,Firth of Forth,Outer Dowsing,400.736837,HVDC_Cable
...,...,...,...,...
230,Nordavind A,NO4,82.890254,HVDC_OffshoreToOnshore
231,Nordavind D,NO4,135.057542,HVDC_OffshoreToOnshore
232,Nordavind C,NO4,139.913997,HVDC_OffshoreToOnshore
233,Vestavind E,NO2,61.196428,HVDC_OffshoreToOnshore


In [150]:
offshr_lines = df_dl[df_dl["LineType"] == "HVDC_Cable"]
offshr_areas = set(offshr_lines["FromNode"].values).union(offshr_lines["ToNode"].values)
offshr_areas

{'Borssele',
 'Dogger Bank',
 'East Anglia',
 'Energyhub EU',
 'Firth of Forth',
 'Helgoländer Bucht',
 'Hollandsee Kust',
 'Hornsea',
 'Moray Firth',
 'Nordavind A',
 'Nordavind B',
 'Nordavind C',
 'Nordavind D',
 'Nordsøen',
 'Nordvest A',
 'Nordvest B',
 'Nordvest C',
 'Norfolk',
 'Outer Dowsing',
 'Sønnavind A',
 'Sørvest A',
 'Sørvest B',
 'Sørvest C',
 'Sørvest D',
 'Sørvest E',
 'Sørvest F',
 'Vestavind A',
 'Vestavind B',
 'Vestavind C',
 'Vestavind D',
 'Vestavind E',
 'Vestavind F'}

Remove cables in original set that are to be replaced 

In [151]:
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeFrom"].isin(offshr_areas)]
directional_lines_tab = directional_lines_tab[~directional_lines_tab["NodeTo"].isin(offshr_areas)].reset_index(drop=True)
directional_lines_tab

,NodeFrom,NodeTo
0,Switzerland,Austria
1,Czech R,Austria
2,Germany,Austria
3,Hungary,Austria
4,Italy,Austria
...,...,...
165,Italy,Slovenia
166,Austria,Switzerland
167,Czech R,Slovakia
168,Hungary,Slovakia


Add newly generated lines (two-way conns)

In [152]:
first_way = df_dl[["FromNode", "ToNode"]].rename(columns={"FromNode": "NodeFrom", "ToNode": "NodeTo"})
second_way = first_way.copy().rename(columns={"NodeFrom": "NodeTo", "NodeTo": "NodeFrom"})
second_way

,NodeTo,NodeFrom
0,Moray Firth,Firth of Forth
1,Firth of Forth,Dogger Bank
2,Firth of Forth,Hornsea
3,Dogger Bank,Hornsea
4,Firth of Forth,Outer Dowsing
...,...,...
230,Nordavind A,NO4
231,Nordavind D,NO4
232,Nordavind C,NO4
233,Vestavind E,NO2


In [153]:
directional_lines_tab = pd.concat([directional_lines_tab, first_way, second_way], ignore_index=True).reset_index(drop=True)
directional_lines_tab

,NodeFrom,NodeTo
0,Switzerland,Austria
1,Czech R,Austria
2,Germany,Austria
3,Hungary,Austria
4,Italy,Austria
...,...,...
635,NO4,Nordavind A
636,NO4,Nordavind D
637,NO4,Nordavind C
638,NO2,Vestavind E


#### Tab: LineTypeOfDirectionalLines

Same structure as Tab: DirectionalLines but with LineType incl. 

In [154]:
line_type_of_directional_lines_tab = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="LineTypeOfDirectionalLines", header=2)
line_type_of_directional_lines_tab

,FromNode,ToNode,LineType
0,Switzerland,Austria,HVAC_OverheadLine
1,Czech R,Austria,HVAC_OverheadLine
2,Germany,Austria,HVAC_OverheadLine
3,Hungary,Austria,HVAC_OverheadLine
4,Italy,Austria,HVAC_OverheadLine
...,...,...,...
419,NO2,Energyhub EU,HVDC_OffshoreToOnshore
420,Great Brit.,Energyhub EU,HVDC_OffshoreToOnshore
421,Netherlands,Energyhub EU,HVDC_OffshoreToOnshore
422,Germany,Energyhub EU,HVDC_OffshoreToOnshore


In [155]:
# Output format with same LineType as before on non-touched lines
df_ltodl = directional_lines_tab.copy().rename(columns={"NodeFrom": "FromNode", "NodeTo": "ToNode"})
df_ltodl = df_ltodl.merge(line_type_of_directional_lines_tab, on=["FromNode", "ToNode"], how="left")
df_ltodl

,FromNode,ToNode,LineType
0,Switzerland,Austria,HVAC_OverheadLine
1,Czech R,Austria,HVAC_OverheadLine
2,Germany,Austria,HVAC_OverheadLine
3,Hungary,Austria,HVAC_OverheadLine
4,Italy,Austria,HVAC_OverheadLine
...,...,...,...
635,NO4,Nordavind A,NaN
636,NO4,Nordavind D,NaN
637,NO4,Nordavind C,NaN
638,NO2,Vestavind E,NaN


In [156]:
df_ltodl.isna().sum()

FromNode      0
ToNode        0
LineType    344
dtype: int64

In [157]:
# LineType of others can be found using df_dl but need to map two-ways

# First-way
offshr_areas_lines = df_dl.copy().drop(columns=["lineLength in km"])
mapping = offshr_areas_lines.set_index(['FromNode', 'ToNode'])['LineType']
df_ltodl['LineType'] = df_ltodl.apply(lambda row: mapping.get((row['FromNode'], row['ToNode']), row['LineType']), axis=1)

# Second-way
offshr_areas_lines = df_dl.copy().drop(columns=["lineLength in km"]).rename(columns={"FromNode": "ToNode", "ToNode": "FromNode"})
mapping = offshr_areas_lines.set_index(['FromNode', 'ToNode'])['LineType']
df_ltodl['LineType'] = df_ltodl.apply(lambda row: mapping.get((row['FromNode'], row['ToNode']), row['LineType']), axis=1)
df_ltodl

,FromNode,ToNode,LineType
0,Switzerland,Austria,HVAC_OverheadLine
1,Czech R,Austria,HVAC_OverheadLine
2,Germany,Austria,HVAC_OverheadLine
3,Hungary,Austria,HVAC_OverheadLine
4,Italy,Austria,HVAC_OverheadLine
...,...,...,...
635,NO4,Nordavind A,HVDC_OffshoreToOnshore
636,NO4,Nordavind D,HVDC_OffshoreToOnshore
637,NO4,Nordavind C,HVDC_OffshoreToOnshore
638,NO2,Vestavind E,HVDC_OffshoreToOnshore


In [158]:
df_ltodl.isna().sum()

FromNode    0
ToNode      0
LineType    0
dtype: int64

In [159]:
line_type_of_directional_lines_tab = df_ltodl.copy()
line_type_of_directional_lines_tab

,FromNode,ToNode,LineType
0,Switzerland,Austria,HVAC_OverheadLine
1,Czech R,Austria,HVAC_OverheadLine
2,Germany,Austria,HVAC_OverheadLine
3,Hungary,Austria,HVAC_OverheadLine
4,Italy,Austria,HVAC_OverheadLine
...,...,...,...
635,NO4,Nordavind A,HVDC_OffshoreToOnshore
636,NO4,Nordavind D,HVDC_OffshoreToOnshore
637,NO4,Nordavind C,HVDC_OffshoreToOnshore
638,NO2,Vestavind E,HVDC_OffshoreToOnshore


#### Tab: GeneratorsOfNode

In [160]:
generators_of_node_tab = pd.read_excel("Data_EMPIRE-Public/Sets.xlsx", sheet_name="GeneratorsOfNode", header=2)
generators_of_node_tab

,Node,Generator
0,Bosnia H,Liginite existing
1,Bulgaria,Liginite existing
2,Czech R,Liginite existing
3,Germany,Liginite existing
4,Estonia,Liginite existing
...,...,...
737,NO1,Hydrogen CCGT
738,NO2,Hydrogen CCGT
739,NO3,Hydrogen CCGT
740,NO4,Hydrogen CCGT


Remove and update offshore wind areas

In [161]:
generators_of_node_tab = generators_of_node_tab[~generators_of_node_tab["Node"].isin(NODES_TO_BE_REMOVED)]
generators_of_node_tab = generators_of_node_tab[~generators_of_node_tab["Node"].isin(offshr_areas)].reset_index(drop=True)
generators_of_node_tab

,Node,Generator
0,Bosnia H,Liginite existing
1,Bulgaria,Liginite existing
2,Czech R,Liginite existing
3,Germany,Liginite existing
4,Estonia,Liginite existing
...,...,...
709,NO1,Hydrogen CCGT
710,NO2,Hydrogen CCGT
711,NO3,Hydrogen CCGT
712,NO4,Hydrogen CCGT


In [162]:
TECH_MAPPING = dict({
    "Wind_offshr_grounded": "Wind offshore grounded",
    "Wind_offshr_floating": "Wind offshore floating"
})

tech_by_offshr_area = pd.read_csv("Output/offshore_capacities_base.csv")[["Node", "GeneratorTechnology"]]
tech_by_offshr_area["GeneratorTechnology"] = tech_by_offshr_area["GeneratorTechnology"].apply(lambda x: TECH_MAPPING.get(x))
tech_by_offshr_area = tech_by_offshr_area.rename(columns={"GeneratorTechnology": "Generator"})
tech_by_offshr_area

,Node,Generator
0,Moray Firth,Wind offshore grounded
1,Firth of Forth,Wind offshore grounded
2,Dogger Bank,Wind offshore grounded
3,Hornsea,Wind offshore grounded
4,Outer Dowsing,Wind offshore grounded
5,Norfolk,Wind offshore grounded
6,East Anglia,Wind offshore grounded
7,Borssele,Wind offshore grounded
8,Hollandsee Kust,Wind offshore grounded
9,Helgoländer Bucht,Wind offshore grounded


In [163]:
generators_of_node_tab = pd.concat([generators_of_node_tab, tech_by_offshr_area], ignore_index=True)
generators_of_node_tab

,Node,Generator
0,Bosnia H,Liginite existing
1,Bulgaria,Liginite existing
2,Czech R,Liginite existing
3,Germany,Liginite existing
4,Estonia,Liginite existing
...,...,...
745,Sørvest A,Wind offshore floating
746,Sørvest D,Wind offshore floating
747,Sørvest E,Wind offshore grounded
748,Sørvest A,Wind offshore grounded


#### Tab: GeneratorsOfTechnology

Keep as is

### Write to Excel and update sheets

Sheets to be updated and corresponding dataframe

In [164]:
SHEET_TO_DF_AND_ROW = dict({
    "Nodes": [nodes_tab, 2],
    "DirectionalLines": [directional_lines_tab, 4],
    "LineTypeOfDirectionalLines": [line_type_of_directional_lines_tab, 4],
    "GeneratorsOfNode": [generators_of_node_tab, 4]
})

In [165]:
# Clean and overwrite existing input_file from original
import shutil

source = "Data_EMPIRE-Public/Sets.xlsx"
destination = "EMPIRE_input/Sets.xlsx"

shutil.copy(source, destination)

'EMPIRE_input/Sets.xlsx'

In [166]:
import openpyxl as ox

def update_spreadsheet(path:str, sheet_name:str, _df : pd.DataFrame, startcol:int=1, startrow:int=4):
    wb = ox.load_workbook(path)
    ws=wb[sheet_name]
    for row in range(0, _df.shape[0]): #For each row in the dataframe
        for col in range(0, _df.shape[1]): #For each column in the dataframe
            ws.cell(row = startrow + row, column = startcol + col).value = _df.iat[row, col]
    wb.save(path)

In [167]:
#Call the defined function
for sheet, df_and_row in SHEET_TO_DF_AND_ROW.items():
    df = df_and_row[0]
    row = df_and_row[1]
    update_spreadsheet(destination, sheet, df, startrow=row)